## Handling States

We first load in our data sets

In [2]:
import numpy as np
import pandas as pd
import os
import math

dtype = {'country_region_code' : object,
'country_region' : object,
'sub_region_1' : object,
'sub_region_2' : object,
'metro_area' : object,
'iso_3166_2_code' : object,
'census_fips_code' : object,
'date' : object,
'retail_and_recreation_percent_change_from_baseline' : float,
'grocery_and_pharmacy_percent_change_from_baseline' : float,
'parks_percent_change_from_baseline	' : float,
'transit_stations_percent_change_from_baseline' : float,
'workplaces_percent_change_from_baseline' : float,
'residential_percent_change_from_baseline' :float,
}

mobFile = 'Global_Mobility_Report.csv'
mobilityDF = pd.read_csv(mobFile, sep=',', error_bad_lines=False, index_col=False, dtype=dtype)
# mobilityDF.head()

policyFile = 'WHO_PHSM_Cleaned_V1_20_09_23.csv'
policyDF = pd.read_csv(policyFile, sep=',', low_memory=False)
# policyDF.head()

casesFile = 'time_series_covid19_confirmed_global.csv'
casesDF = pd.read_csv(casesFile, sep=',', low_memory=False)
#casesDF.head()

deathsFile = 'time_series_covid19_deaths_global.csv'
deathsDF = pd.read_csv(deathsFile, sep=',', low_memory=False)
# deathsDF.head()

Then we begin by defining some useful variables in creating our dataset

In [3]:
states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

memo = {
    'new': 1,
    'extension': 1,
    'modification': 0,
    'finish': -1, 
    'phase-out': -1, 
}

dates = ['1/1/2020','1/2/2020','1/3/2020','1/4/2020','1/5/2020','1/6/2020','1/7/2020'
         ,'1/8/2020','1/9/2020','1/10/2020','1/11/2020',
         '1/12/2020','1/13/2020','1/14/2020','1/15/2020','1/16/2020',
         '1/17/2020','1/18/2020','1/19/2020','1/20/2020','1/21/2020']
dates += casesDF.columns.tolist()[4:]
print('Period is: ', dates[0], '-',dates[-1])

Period is:  1/1/2020 - 11/9/2020


### Statewide policy Data

For each of the states, we want to create a memo as we did last ime

In [4]:
US_Policy = policyDF[ policyDF['country_territory_area'] == 'United States Of America']
whoCodes = whoCodes = sorted(list(policyDF.sort_values(by='who_code').who_code.unique()))

stateData = {}
usData = {}

for state in states:
    stateData[state] = {}
    for date in dates:
        stateData[state][date] = {}
        usData[date] = {}
        for code in whoCodes:
            stateData[state][date][code] = 0
            usData[date][code] = 0        

We increment the national policy and the us policies on the date that they are added to the memo. 

In [5]:
for policy in US_Policy.to_dict(orient='records'):
    (country, who_code, date_start, date_end, admin_level, measure_stage) = (policy['country_territory_area'], policy['who_code'],policy['date_start'], policy['date_end'], policy['admin_level'], policy['measure_stage'])
    who_measure = policy['who_measure']
    area_covered = policy['area_covered']
    
    if admin_level == 'national':
        if isinstance(date_start, str) and isinstance(who_code, str) and measure_stage != 'modification' and usData[date_start][who_code] != 1:
            usData[date_start][who_code] = memo[measure_stage]
    if admin_level == 'state' and area_covered in states:
        if isinstance(date_start, str) and isinstance(who_code, str) and measure_stage != 'modification' and stateData[area_covered][date_start][who_code] != 1:
            stateData[area_covered][date_start][who_code] = memo[measure_stage]


We then normalize our values by taking a running count of whether the policy is active or not. 1's Denote the addtion of a policy, 0's denote no action, and -1's denote removal of a policy

In [6]:
for state in states:
    for i in range(1, len(dates)):
        for key in whoCodes:
            if stateData[state][dates[i]][key] == 0:
                stateData[state][dates[i]][key] = stateData[state][dates[i-1]][key]
            elif stateData[state][dates[i]][key] == 1:
                stateData[state][dates[i]][key] = 1
            elif stateData[state][dates[i]][key] == -1:
                stateData[state][dates[i]][key] = 0 
                
for i in range(1, len(dates)):
    for key in whoCodes:
        if usData[dates[i]][key] == 0:
            usData[dates[i]][key] = usData[dates[i-1]][key]
        if usData[dates[i]][key] == 1:
            usData[dates[i]][key] = 1
        if usData[dates[i]][key] == -1:
            usData[dates[i]][key] = 0

In [7]:
for state in states:
    for i in range(1, len(dates)):
        for key in whoCodes:
            if usData[dates[i]][key] == 1:
                stateData[state][dates[i]][key] = 1

### Statewide case numbers

Case numbers are loaded in from our cases time series data. For each state we get cumulative cases on each day

In [8]:
filename = './COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
casesDF = pd.read_csv(filename, sep=',',error_bad_lines=False, index_col=False, dtype=dtype)

dates = casesDF.columns.tolist()[11:]
    
for state in states:
    for date in dates:
        stateData[state][date]['totalCases'] = 0
        

for index, row in casesDF.iterrows():
    state = row['Province_State']
    if state not in states:
        continue
    for date in dates:
        stateData[state][date]['totalCases'] += row[date]
        

Here we calculate the active cases on each date. From these active case numbers, we get a growth rate, as well as a change in the growth rate. 

In [9]:
offset = 5 #offset for number of days we use for covid symptoms to show up 
diff = 5 # Diff for number of dates in which we use to calculate growth
symLen = 14 # length of symptoms
for state in states:
    for i in range(symLen + offset, len(dates) - offset - diff):
        activeCasesOld = stateData[state][dates[i-diff+offset]]['totalCases'] - stateData[state][dates[i-diff+offset - symLen]]['totalCases']
        activeCases = stateData[state][dates[i+offset]]['totalCases'] - stateData[state][dates[i+offset - symLen]]['totalCases']
        activeCasesNext = stateData[state][dates[i+diff+offset]]['totalCases'] - stateData[state][dates[i+diff+offset - symLen]]['totalCases']

        if activeCasesOld < 100:
            continue
        
        oldGrowth = (activeCases / activeCasesOld) ** (1 / diff) if activeCasesOld > 0 else -1
        growth = (activeCasesNext / activeCases) ** (1/diff) if activeCases > 0 else -1
        
        stateData[state][dates[i]]['growth'] = growth
        stateData[state][dates[i]]['oldGrowth'] = oldGrowth
        stateData[state][dates[i]]['flattening'] = 1 if math.log(growth / oldGrowth, 2) < 0 else 0
        stateData[state][dates[i]]['activeCases'] = activeCases
        


### Statewide mobility numbers

Google mobility data was loaded in. m2 m4 m5 were deemed essentail mobility, while m1 m3 and m6 were deemed non-essential mobility. 

In [10]:
usMobilityDF = pd.read_csv('./Region_Mobility_Report_CSVs/' + '2020_US_Region_Mobility_Report.csv', sep=',', error_bad_lines=False, index_col=False, dtype=dtype)

def revDay(filename):
    # 04-12-2020
    if filename[-4:] == ".csv":
        filename = filename[:-4]
    (year, month, day) = filename.split('-')
    if(month[0] == '0'):
        month = month[1:]
    if(day[0] == '0'):
        day = day[1:]
    return '/'.join( [month, day, year] )    

stateDF = usMobilityDF[ pd.isnull(usMobilityDF['sub_region_2'])]
i = 0

for index, row in stateDF.iterrows():
    country = row['country_region']
    state = row['sub_region_1']
    date = revDay(row['date'])

    mob = {
        'm1': row['retail_and_recreation_percent_change_from_baseline'],
        'm2': row['grocery_and_pharmacy_percent_change_from_baseline'],
        'm3': row['parks_percent_change_from_baseline'],
        'm4': row['transit_stations_percent_change_from_baseline'],
        'm5': row['workplaces_percent_change_from_baseline'],
        'm6': row['residential_percent_change_from_baseline'],
    }
    if country == "United States" and not isinstance(state, str):
        if date not in usData: 
            continue
        usData[date]['Essential'] = 1.0 + ((mob['m2'] + mob['m4'] + mob['m5']) / 300 )
        usData[date]['nonEssential'] = 1.0 + ( (mob['m1'] + mob['m6']) / 300 )
    if country == "United States" and state in states:
        if date not in stateData[state]:
            continue
        stateData[state][date]['Essential'] = 1.0 + ((mob['m2'] + mob['m4'] + mob['m5']) / 300 )
        stateData[state][date]['nonEssential'] = 1.0 + ( (mob['m1'] + mob['m6'] + mob['m3']) / 300 )
        #for k,v in mob.items():
        #   stateData[state][date][k] = 1.0 + v / 100
            
    
    i += 1
    if i % 1000 == 0:
        print(i)
    

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000


This was then written to our file.

In [11]:
headers = 'state,date,' + ','.join(whoCodes) + ',totalCases,growth,oldGrowth,flattening,activeCases,essential,nonessential\n' 

outFile = open('US-Data-Final.csv', 'w')
outFile.write(headers)

def toStr(x):
    return str(x)

for (state, obj1) in stateData.items():
    outMemo = {'state': state}
    for (date, obj2)  in obj1.items():
        outMemo['date'] = date
        if len(obj2) != 52:
            continue
        for (code, val) in obj2.items():
            outMemo[code] = val
        vals = ','.join( map(toStr ,list(outMemo.values())))
        vals += ',\n'
        outFile.write(vals)

outFile.close()